In [ ]:
import tensorflow as tf
import numpy      as np

import directoryFunctions
import pathlib
import random
import data
import time

from tensorflow.keras.optimizers  import Adam
from tensorflow.keras.callbacks   import CSVLogger, EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.losses      import CategoricalCrossentropy
from tensorflow.keras.layers      import Dense, Dropout, LSTM
from tensorflow.keras.models      import load_model, Sequential

In [ ]:
def getSequences(datasetType):
    maxFrameCount = dataObj.getMaxFrameCount()
    sequences = []
    labels    = []
    dataRows = dataObj.data[:]
    random.shuffle(dataRows)
    sequencesPath = pathlib.Path(r"D:\ActionRecognition\Sequences")
    for dataRow in dataRows:
        if(datasetType == dataRow[0]):
            sequencePath = sequencesPath/dataRow[0]/dataRow[1]/(dataRow[2] + "_featureSequence.npy")
            sequence     = np.load(sequencePath)
            sequence     = np.pad(sequence, ((0, maxFrameCount - int(dataRow[3])), (0, 0)), 'edge')
            sequences.append(sequence)
            label = dataObj.getClassIndex(dataRow[1])
            labels.append(label)
    return np.array(sequences), np.array(labels, dtype=np.uint8)

In [ ]:
def getDataset(sequences, labels):
    dataset = tf.data.Dataset.from_tensor_slices((sequences, labels))
    return dataset

In [ ]:
def prepareTrainDataset(dataset, cache, shuffleBufferSize):
    if cache:
        if isinstance(cache, str):
            dataset = dataset.cache(cache)
        else:
            dataset = dataset.cache()
    
    dataset = dataset.shuffle(buffer_size = shuffleBufferSize)
    # Repeat forever
    dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE)
    # `prefetch` lets the dataset fetch batches in the background while the model is training.
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset

In [ ]:
def getModel():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(256, input_shape=(None, 2048)))
    model.add(tf.keras.layers.Dense(64, activation = 'relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(numClasses, activation = 'softmax')) # change 3 to numClasses
    
    model.compile(optimizer = tf.keras.optimizers.Adam(lr = 0.0001),
                  loss      = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                  metrics   = ['accuracy'])
    return model

In [ ]:
def trainModel(model, epochs, trainDataset, validationDataset, steps_per_epoch, validation_steps, callbacks):
    history = model.fit(trainDataset, 
                        epochs = epochs,
                        validation_data  = validationDataset,
                        steps_per_epoch  = steps_per_epoch,
                        validation_steps = validation_steps,
                        callbacks = callbacks)
    return model, history

In [ ]:
dataObj    = data.Data()
numClasses = dataObj.numClasses
AUTOTUNE   = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

In [ ]:
def main():
    epochs = 5
    
    cachePath     = pathlib.Path(r"./Cache")
    cacheFilePath = cachePath/'trainRNNDatasetCache'
    directoryFunctions.removeDirectory(cachePath)
    directoryFunctions.createDirectory(cachePath)
    
    rootPath = pathlib.Path(r"D:\ActionRecognition")
    rnnCallbacksDirectory = rootPath/'Callbacks'/'RNN'/f'{numClasses}'
    
    trainSeqCount      = dataObj.getDatasetCount('Train')
    validationSeqCount = dataObj.getDatasetCount('Validation')
    
    trainSequences, trainLabels           = getSequences('Train')
    validationSequences, validationLabels = getSequences('Validation')
    
    trainDataset      = getDataset(trainSequences, trainLabels)
    validationDataset = getDataset(validationSequences, validationLabels)
    
    trainDataset      = prepareTrainDataset(trainDataset, str(cacheFilePath), trainSeqCount)
    validationDataset = validationDataset.batch(BATCH_SIZE)
    
    steps_per_epoch  = np.ceil(trainSeqCount/BATCH_SIZE)
    validation_steps = np.ceil(validationSeqCount/BATCH_SIZE)
    
    modelCheckpointDirectory = rnnCallbacksDirectory/'ModelCheckpoint'
    tensorboardDirectory     = rnnCallbacksDirectory/'Tensorboard'
    csvLoggerDirectory       = rnnCallbacksDirectory/'CSVLogger'
    
    directoryFunctions.createDirectory(csvLoggerDirectory)
    directoryFunctions.createDirectory(modelCheckpointDirectory)
    
    modelCheckpoint = ModelCheckpoint(filepath       = str(modelCheckpointDirectory/(f'{int(time.time())}' + '_RNN_{epoch:03d}_{val_loss:.2f}.h5')),
                                      save_best_only = True)
    tensorboard     = TensorBoard(log_dir = str(tensorboardDirectory/f'{int(time.time())}'))
    csvLogger       = CSVLogger(str(csvLoggerDirectory/f'{int(time.time())}.log'))
    earlyStopping   = EarlyStopping(monitor = 'val_loss', patience = 5)
    callbacks       = [modelCheckpoint, tensorboard, csvLogger, earlyStopping]
    
    savedModelPath = "" # insert path to saved model (.h5 file) here
    if savedModelPath == "":
        model = getModel()
    else:
        model  = load_model(savedModelPath)
        epochs = 5
    
    trained_model, history = trainModel(model, epochs, 
                                        trainDataset, validationDataset, 
                                        steps_per_epoch, validation_steps, callbacks)
    
    directoryFunctions.removeDirectory(cachePath)

In [ ]:
main()